In [16]:
from langgraph.graph import MessagesState
import pandas as pd

class State(MessagesState):
    subject: str = ''
    description: str = ''
    data: pd.DataFrame = pd.DataFrame()
    
    relevant_data: pd.DataFrame = pd.DataFrame()
    

In [17]:
# Schema
from typing_extensions import Annotated
from pydantic import Field, BaseModel

class IDRelevance(BaseModel):
    relevant_id: Annotated[
        list[int],
        Field(
            ..., 
            max_items=5, 
            min_items=3,
            description="가장 관련성이 높은 데이터 ID 리스트, 길이 최소 3개/최대 5개", 
        )
    ]

In [18]:
# 프롬프트
from langchain_core.prompts import PromptTemplate

relevance_template = '''
당신은 데이터 과학자입니다. 아래는 연구 데이터 목록입니다.

각 데이터 항목은 다음 컬럼을 가지고 있습니다:
- ID: 각 데이터의 고유키
- 제목
- 설명
- 키워드

[목표]
주어진 연구 주제와 가장 관련성 높은 3~5개의 데이터 항목을 선택하세요. 

[Input]
연구 주제: {subject}
연구 설명: {description}

[Data]
데이터 목록:
{data}

[Output]
가장 관련성 높은 3~5개 항목의 ID를 JSON으로 출력해주세요.
'''

relevance_prompt = PromptTemplate.from_template(relevance_template)


In [19]:
from langchain_openai import ChatOpenAI

def select_relevance(state: State):

    df = state['data']

    prompt = relevance_prompt.invoke(
        {
            'subject': state['subject'], 
            'description': state['description'],
            'data': df[['ID', '제목', '설명', '키워드']].to_dict(orient="records"),
        }
    )

    # sLLM
    sllm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

    structured_sllm = sllm.with_structured_output(IDRelevance)
    res = structured_sllm.invoke(prompt)

    # Output
    id_list = res.relevant_id
    selected_data = df[df['ID'].isin(id_list)]

    return {'relevant_data':selected_data}


In [20]:
from langgraph.graph import START, END, StateGraph

def build_graph():
    builder = StateGraph(State)
    
    builder.add_node('select_relevance', select_relevance) 
    
    builder.add_edge(START, 'select_relevance')
    builder.add_edge('select_relevance', END)
    
    return builder.compile()

graph = build_graph()

In [21]:
import pandas as pd

subject = '기후 데이터 기반의 도시 열섬 현상 분석'
description = '위성 원격탐사 데이터와 국내 기상 관측소 데이터를 결합하여 도시 열섬 현상의 시공간적 패턴을 분석하고, 인공지능 기반 예측 모델을 통해 기후 변화 대응 전략을 모색하는 연구'
processed_df = pd.read_csv('./generated_input_data.csv', encoding='UTF-8', low_memory=False)

res = graph.invoke({
    'subject': subject,
    'description': description,
    'data': processed_df, 
})

display(res['relevant_data'])

,ID,구분,제목,설명,작성일,해외/국내,작성언어,키워드
13,14,논문,기후과학을 위한 효율적 알고리즘 연구,기후과학 분야에서 성능과 계산비용을 동시에 개선한 새로운 알고리즘을 제안하고 실험으...,2017-07-25,국내,한국어,"기후과학, 알고리즘, 성능평가, 실험"
24,25,데이터셋,Climate Science Public Dataset (multi-country),A large labeled dataset for Climate Science co...,2023-07-08,해외,English,"Climate Science, dataset, labeling, preprocessing"
27,28,논문,A scalable method for Climate Science,This paper presents a scalable method that imp...,2023-11-25,해외,English,"Climate Science, algorithm, benchmark, ablation"
50,51,데이터셋,기후과학 공개 데이터셋 (국내 수집),국내 환경에서 수집된 기후과학 관련 대규모 라벨링 데이터셋입니다. 메타데이터와 전처...,2021-01-13,국내,한국어,"기후과학, 데이터셋, 라벨링, 전처리"
58,59,논문,A scalable method for Climate Science,This paper presents a scalable method that imp...,2025-09-19,해외,English,"Climate Science, algorithm, benchmark, ablation"


In [33]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document

# 1. 텍스트 결합
vector_df = processed_df[['ID', '제목', '설명', '키워드']]

texts = (vector_df['제목'] + " " +
         vector_df['설명'] + " " +
         vector_df['키워드']).tolist()

# 2. 임베딩 객체 생성
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 3. Document 객체로 변환 (FAISS용)
docs = [Document(page_content=text, metadata={"id": row.ID}) 
        for text, row in zip(texts, vector_df.itertuples())]

# 4. FAISS 벡터스토어 생성
vectorstore = FAISS.from_documents(docs, embeddings)

# 5. 쿼리 임베딩 생성
query = subject + " " + description
query_embedding = embeddings.embed_query(query)

# 6. 유사 문서 검색
results_with_score = vectorstore.similarity_search_with_score_by_vector(query_embedding, k=20)

# 7. 결과 출력
df_results = pd.DataFrame([
    {
        "ID": r.metadata.get("id"),
        "제목": vector_df.loc[vector_df['ID'] == r.metadata.get("id"), "제목"].values[0],
        "설명": vector_df.loc[vector_df['ID'] == r.metadata.get("id"), "설명"].values[0],
        "유사도": score,
    }
    for r, score in results_with_score
]).sort_values(by="유사도", ascending=False)

display(df_results)

,ID,제목,설명,유사도
19,35,생물정보학 공개 데이터셋 (국내 수집),국내 환경에서 수집된 생물정보학 관련 대규모 라벨링 데이터셋입니다. 메타데이터와 전...,1.372959
18,20,자연어처리 공개 데이터셋 (국내 수집),국내 환경에서 수집된 자연어처리 관련 대규모 라벨링 데이터셋입니다. 메타데이터와 전...,1.361324
17,2,A scalable method for Remote Sensing,This paper presents a scalable method that imp...,1.334472
16,25,Climate Science Public Dataset (multi-country),A large labeled dataset for Climate Science co...,1.332767
15,60,A scalable method for Social Science,This paper presents a scalable method that imp...,1.318056
14,45,컴퓨터비전 공개 데이터셋 (국내 수집),국내 환경에서 수집된 컴퓨터비전 관련 대규모 라벨링 데이터셋입니다. 메타데이터와 전...,1.312373
11,11,원격탐사 공개 데이터셋 (국내 수집),국내 환경에서 수집된 원격탐사 관련 대규모 라벨링 데이터셋입니다. 메타데이터와 전처...,1.283257
12,42,원격탐사 공개 데이터셋 (국내 수집),국내 환경에서 수집된 원격탐사 관련 대규모 라벨링 데이터셋입니다. 메타데이터와 전처...,1.283257
13,44,원격탐사 공개 데이터셋 (국내 수집),국내 환경에서 수집된 원격탐사 관련 대규모 라벨링 데이터셋입니다. 메타데이터와 전처...,1.283257
10,17,그래프분석 공개 데이터셋 (국내 수집),국내 환경에서 수집된 그래프분석 관련 대규모 라벨링 데이터셋입니다. 메타데이터와 전...,1.267135
